K-Means Clustering with Toronto Neighborhoods 

Begin by assigning beautiful soup object to page contents of wikipedia page

In [7]:
from bs4 import BeautifulSoup
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

create list of table contents by parsing table out of page contents while excluding rows with unassigned Borough

In [10]:
import pandas as pd

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


create pandas dataframe from list and rename boroughs.

In [11]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

print(df)

    PostalCode                Borough  \
0          M3A             North York   
1          M4A             North York   
2          M5A       Downtown Toronto   
3          M6A             North York   
4          M7A           Queen's Park   
..         ...                    ...   
98         M8X              Etobicoke   
99         M4Y       Downtown Toronto   
100        M7Y  East Toronto Business   
101        M8Y              Etobicoke   
102        M8Z              Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4                        Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Welles

Append spatial coordinates for each postal code

In [23]:
coords_df = pd.read_csv(r'C:\Users\jackh\Desktop\projects\IBM-Applied-Data-Science-Capstone\data\Geospatial_Coordinates.csv')
toronto_df = pd.merge(left=df, right=coords_df, left_on='PostalCode', right_on='Postal Code').drop('Postal Code', axis=1)
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


Perform Visualization and Analysis

In [25]:
print('Our Toronto DataFrame has {} Boroughs and {} Neighborhoods.'.format(
    len(toronto_df['Borough'].unique()),
    len(toronto_df['Neighborhood'].unique())
))

Our Toronto DataFrame has 15 Boroughs and 103 Neighborhoods.


Find coordinates for city of Toronto to define map view

In [26]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent='toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'Latitude: {latitude}, Longitude: {longitude}')

Latitude: 43.6534817, Longitude: -79.3839347


Initialize map and add markers for each neighborhood

In [ ]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)

map_toronto